# Aggregation with dynamic features
This notebook shows how to do the aggregation with monthly values for certain columns. 

__Remark:__ Because we now filter out a lot of users that only visited once, this notebook is not such a pain in the ass anymore. Don't be afraid to run it, your memory will be sufficient and you'll be done in a couple of minutes.

In [ ]:
import json
import datetime
import os
import time
import sys
import shutil
import glob
import re

import pandas as pd
import numpy as np
from sklearn import preprocessing

import matplotlib.pyplot as plt

sys.path.append('..')
from preprocessing import *
from aggregation import *

## Managing a huge file

Below is the new version of `load`, where processing takes place in chunks. After all chunks have been processed, they are concatenated to a single file. Since many columns are either dropped or aggregated, the resulting dataframe fits in RAM.

In [ ]:
# Only run these the first time - after it you can just load the reduced_datasets.
reduce_df("../data/train_v2.csv", output="../data/reduced_train.csv", nrows=None, chunksize=20000)
reduce_df("../data/test_v2.csv", output="../data/reduced_test.csv", nrows=None, chunksize=20000)

In [ ]:
# print("Let's widen the train dataset.")
train = pd.read_csv("../data/reduced_train.csv", dtype={'fullVisitorId': 'str'})
wide_train = aggregate_only(train)
wide_train.to_csv("../data/wide_train.csv", encoding="utf-8", index=False)

print("Let's widen the test dataset.")
test = pd.read_csv("../data/reduced_test.csv", dtype={'fullVisitorId': 'str'})
wide_test = aggregate_only(test)
wide_test.to_csv("../data/wide_test.csv", encoding="utf-8", index=False)

## Here the dataset with the static variables as HOE merged with the dynamic variables



In [ ]:
# load external data

train_e = pd.read_csv("../data/wide_train_with_external_data.csv", dtype={'fullVisitorId': 'str'}) 
test_e = pd.read_csv("../data/wide_test_with_external_data.csv", dtype={'fullVisitorId': 'str'}) 
def aggregate_data(train_e, test_e):
    names = ['fullVisitorId', 'GDP', 'HDI']
    train_e = train_e[names]
    test_e = test_e[names]
    merged_e = pd.concat([train_e, test_e], sort=False)
    def trans_to_float(x):
        try:
            return float(x)
        except ValueError:
            return np.nan
    
    merged_e['HDI'] = merged_e['HDI'].apply(lambda x: trans_to_float(x))
    print(merged_e.columns)
    print(merged_e.dtypes)
    merged_e =  merged_e.groupby("fullVisitorId", as_index = False).mean()
    print(merged_e.columns)
    return merged_e

external_data = aggregate_data(train_e, test_e)
external_data.to_pickle('../data/external_data.pkl')
train_e, test_e = None, None

In [ ]:
def stack_google(google, test, date_from=('16-10-2018')):
    print('Adding the Google columns')
    google['date'] = google['date'].apply(lambda x: pd.datetime.strptime(x, '%Y-%m-%d'))
    test['date'] = test['date'].apply(lambda x: pd.datetime.strptime(x, '%Y-%m-%d'))
    google = google[google['date']>=date_from]
    google.columns = ['Unnamed: 0', 'Client ID', 'sessions', 'Avg. Session Duration',
       'Bounce Rate', 'transactions', 'date', 'transactionRevenue', 'fullVisitorId',
       'date_month', 'date_week']
                 
    for iter_ in test.columns:
        try:
            a = google[iter_]
        except:
            print(iter_)
            google[iter_] = 0
    
    google = google[test.columns]
    return pd.concat([test, google], sort=False)

def merge_google(df, google):
        def to_date(x):
            try:
                return pd.datetime.strptime(x, '%Y-%m-%d')
            except TypeError:
                return x
            
        google['date'] = google['date'].apply(lambda x: to_date(x))
        df['date'] = df['date'].apply(lambda x: to_date(x))
        google = google[['fullVisitorId', 'date', 'Avg. Session Duration', 'Bounce Rate']]
        return df.merge(google, on=['fullVisitorId', 'date'], how ='left')
    

In [ ]:
test = pd.read_csv("../data/reduced_test.csv", dtype={'fullVisitorId': 'str'})
train = pd.read_csv("../data/reduced_train.csv", dtype={'fullVisitorId': 'str'})
google = pd.read_csv("../data/google_analytics.csv", dtype={'fullVisitorId': 'str'})

test = stack_google(google, test)
    
print('Merging the Google columns')
train = merge_google(train, google)
test = merge_google(test, google)


train.to_pickle('../data/train.pkl')
test.to_pickle('../data/test.pkl')

In [ ]:
# # let's merge the datasets
train = pd.read_pickle("../data/train.pkl")
test = pd.read_pickle("../data/test.pkl")

merged  = deal_static(train, 
                      test, 
                      selec_top_per=0.5,
                      max_cat=10)

train, test = None, None  # To liberate memory space

merged.to_pickle("../data/merged.pkl")

### Change the date period!

Once you have the merged dataset, you only need to run this chunck of code and change the date

In [ ]:
merged = pd.read_pickle("../data/merged.pkl")
external_data = pd.read_pickle('../data/external_data.pkl')

merged = merged.merge(external_data, how='left', on = 'fullVisitorId')
# Date intervals to split the data
x_train_dates=('2016-08-01', '2017-11-30') 
y_train_dates=('2017-12-01', '2018-01-31')
x_test_dates=('2017-08-01', '2018-11-30')

# Final data processing
x_train, y_train, x_test, y_test = create_train_test(merged, 
                                                     x_train_dates=x_train_dates, 
                                                     y_train_dates=y_train_dates, 
                                                     x_test_dates=x_test_dates, 
                                                     y_test_dates=None)

# Save dfs as pickle objects -> faster to load and save. In addition, we do not need to worry about format issues
x_train.to_pickle("../data/x_train.pkl") 
y_train.to_pickle("../data/y_train.pkl") 
x_test.to_pickle("../data/x_test.pkl")

if y_test is not None:
    y_test.to_pickle("../data/y_test.pkl")

### Our first attempt!
Let's see if we can fit a model on this data.

Note that due to the one-hot encoding, the columns of train and test are not the same. For this experiment, only keep the intersection of columns. There are also other ways to deal with this (e.g., by mapping categories to external data), so we don't do this in the aggregation step.

In [ ]:
x_train = pd.read_pickle("../data/x_train.pkl") 
y_train = pd.read_pickle("../data/y_train.pkl") 
x_test = pd.read_pickle("../data/x_test.pkl") 

# Save the dataset ids
id_x_train =  x_train['fullVisitorId']
id_y_train = y_train['fullVisitorId']
id_x_test = x_test['fullVisitorId']

# Delete fullVisitor ID -> probably we want to leave it as a OHE feature
del x_train['fullVisitorId']
del y_train['fullVisitorId']
del x_test['fullVisitorId']


In [ ]:
# set NaNs to zero and fit linear model
from sklearn import linear_model
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)
y_train = y_train.fillna(0)

lm = linear_model.LinearRegression()
lm.fit(x_train, y_train)
r_squared = lm.score(x_train, y_train)
print("The model has an R^2 of {}.".format(r_squared))
# do the prediction
prediction = list(lm.predict(x_test).flat)